In [ ]:
%load_ext autoreload
%autoreload 2
from os.path import join
from tqdm import tqdm
import pandas as pd
import numpy as np
import clin.llm
import clin.eval.med_status
import clin.parse
import openai
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'
from typing import List
results_dir = '../results/'
from clin.config import PATH_REPO
import datasets
import imodelsx.process_results
def viz_blue_rows(df):
    return (
        df
        # highlight rows 2 and 4 including the index
        .style.apply(lambda x: ['background: lightblue' if 'self-verified' in x.name else '' for i in x], axis=1)
        # bold index label for row 2
        .apply(lambda x: ['font-weight: bold' if 'self-verified' in x.name else '' for i in x], axis=1)
        .format(precision=3)
    )   
r = imodelsx.process_results.get_results_df(results_dir, use_cached=False)

In [ ]:
r_ebm = r[r.dataset_name == 'ebm']
cols = {
    'f1___original': 'Clinical trial arm extraction, original',
    'f1___ov_pv_ev': 'Clinical trial arm extraction, self-verified',
}
rt_ebm = r_ebm.groupby(['checkpoint', 'n_shots'])[list(cols.keys())].mean().rename(columns=cols).round(3).T
viz_blue_rows(rt_ebm)

In [ ]:
r_med = r[r.dataset_name == 'medication_status']
dset = datasets.load_dataset('mitclinicalml/clinical-ie', 'medication_status')
df_val = pd.DataFrame.from_dict(dset['validation'])
df = pd.DataFrame.from_dict(dset['test'])
# df = pd.concat([val, test])
nums = np.arange(len(df)).tolist()
np.random.default_rng(seed=13).shuffle(nums)
dfe = df.iloc[nums]
r_med = clin.eval.med_status.add_status_eval(r_med, dfe)

cols = {
    'f1___original': 'Medication extraction, original',
    'f1___ov_pv_ev': 'Medication extraction, self-verified',
    'status_f1_macro_cond___original': 'Medication status, original',
    'status_f1_macro_cond___sv': 'Medication status, self-verified',
}
rt_med_status = r_med.groupby(['checkpoint', 'n_shots'])[list(cols.keys())].mean().rename(columns=cols).round(3).T
viz_blue_rows(rt_med_status)

In [ ]:
rt = pd.concat((rt_ebm, rt_med_status))
viz_blue_rows(rt)